In [2]:
import pandas as pd

In [10]:
parts = ["back", "leg", "arm", "seat"]
def get_part(x):
    for part in parts:
        if part in x:
            return part
    raise
df = pd.read_csv('/scratch/noam/control_point_e/datasets/chair_llama3_not_unk_top100/val.csv')
df = df[df.llama3_utterance != "Unknown"]
df["part"] = df.llama3_utterance.apply(lambda x: get_part(x))
df.to_csv('/scratch/noam/control_point_e/datasets/chair_llama3_not_unk_top100/val.csv', index=False)

In [8]:
df.columns

Index(['workerid', 'prompt', 'utterance', 'assignmentid', 'worktimeinseconds',
       'source_model_name', 'source_object_class', 'source_dataset',
       'target_model_name', 'target_object_class', 'target_dataset',
       'is_patched', 'target_uid', 'source_uid', 'hard_context',
       'target_original_object_class', 'source_original_object_class',
       'saliency', 'tokens', 'tokens_len', 'utterance_spelled',
       'target_unary_split', 'source_unary_split', 'listening_split',
       'changeit_split', 'tokens_encoded', 'llama3_utterance',
       'source_wnlemmas', 'target_wnlemmas', 'intersect_wnlemmas',
       'random_wnlemma', 'llama3_wnlemma_utterance', 'chamfer_distance',
       'part'],
      dtype='object')

In [18]:
import shutil
html = "<table>\n"
html += "<tr><th>index<th/><th>utterance<th/><th>source<th/><th>target<th/></tr>\n"
for i, row in df.iterrows():
    source_image = f"/scratch/noam/shapetalk/images/full_size/{row.source_uid}.png"
    target_image = f"/scratch/noam/shapetalk/images/full_size/{row.target_uid}.png"
    shutil.copy(source_image, f"/home/noamatia/repos/point-e/output/images/{row.source_uid}.png")
    shutil.copy(target_image, f"/home/noamatia/repos/point-e/output/images/{row.target_uid}.png")
    html += f"<tr><td>{i}</td><td>{row.utterance}</td><td><img src='images/{row.source_uid}.png'></td><td><img src='images/{row.target_uid}.png'></td></tr>\n"
html += "</table>"
with open("/home/noamatia/repos/point-e/output/index.html", "w") as f:
    f.write(html)


In [14]:
df.shape

(62392, 33)

In [15]:
df.index.unique()

RangeIndex(start=0, stop=62392, step=1)

In [ ]:
630
12094
4589
11302
24905
12230
35315
34940
2557
22293
9605
11379
12192
62001
12309
48829
23855
36353
12469
9747
2190
12538
45809


In [ ]:
import torch
from tqdm.auto import tqdm

from point_e.diffusion.configs import DIFFUSION_CONFIGS, diffusion_from_config
from point_e.diffusion.sampler import PointCloudSampler
from point_e.models.download import load_checkpoint
from point_e.models.configs import MODEL_CONFIGS, model_from_config
from point_e.util.plotting import plot_point_cloud

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('creating base model...')
base_name = 'base40M-textvec'
base_model = model_from_config(MODEL_CONFIGS[base_name], device)
base_model.create_control_layers()